#A/B тестирование

Предположим, что нам нужно произвести результаты независимого A/B тестирования в котором мы хотим проверить гипотезу о том, что ручное внесение количества прибывшего товара занимает столько же времени как и сканирование этого товара.


- H0 - среднее время действий разных видов (сканирование или ручное внесение) равны
- H1 - среднее время разных видов действий - различное.


С гипотезой определились, теперь нам нужно понять размер выборки для проверки.
Наше тестирование длилось месяц, поэтому есть всего месяц данных, а генеральная совокупность - все сканирования или ручные внесения происходящие при приеме товара. Т.е. наша ГС - гораздо больше чем 1 месяц, а значит наши данные - выборка из ГС.


С размером выборки определились и влетели с двух ног в новый вопрос: "какой тест для проверки гипотезы использовать? Можем попробовать t-критерий Стьюдента или же тест Манна-Уитни. Первый параметрический, второй - нет." 


Для ответа на этот вопрос нужно вспомнить критерии применимости t-критерия: 
- некоррелируемость признаков
- нормальность распределения признаков 
- гомогенность дисперсий признаков. 


С первым пунктом все понятно - действия независимые и время сканирования никак не влияет на время ручных корректировок или влияет очень опосредованно, например когда тупит комп - время и ручных внесений и сканирований увеличиваются. Но мы этим можем пренебречь. А вот второй и третий критерий применимости t-критерия - нужно проверять. 


Для проверки на нормальность можно использовать, например тест Шапиро-Вилка. При этом, нужно понимать, что даже отвергание гипотезы о нормальности распределения не дает однозначного запрета на использование критерия Стьюдента потому что доказано, что при больших выборках (где n>30) требование нормальности распределения для t-критерия  является необязательным.


При проверрке на гомогенность дисперсии можно использовать критерий Фишера. Тут важно помнить, что в отличае от t-теста, для F-критерия требование нормальности распределений является обязательнымзатем и, в случае нарушения этого требования, можно легко, без регистраций и смс, получить неадекватные результаты. Поэтому, если вдруг наши выборки распределены не нормально, то следует использовать какой-либо непараметрический аналог, например тест Крускала-Виллиаса (он, кстати, хорошо работает если выборки разного размера). 


И только после всех проверок можно однозначно сказать, какой именно критерий для проверки гипотез нам следует применять (Стьюдента или Манна-Уитни), применить и сделать выводы.

In [ ]:
from scipy import stats as st
import numpy as np
import pandas as pd


sample_1 = df.query('ScanDuration != 0')['ScanDuration'].values
sample_2 = df.query('HandDuration != 0')['HandDuration'].values

# критический уровень статистической значимости для гипотезы
alpha = 0.05  


# проверим распределения на нормальность
shapiro_1 = st.shapiro(sample_1).pvalue
shapiro_2 = st.shapiro(sample_2).pvalue

# проверим гомогенность дисперсии
if shapiro_1 >= alpha and shapiro_2 >= alpha:

    F = np.var(sample_1) / np.var(sample_2)
    df_1 = len(sample_1)-1
    df_2 = len(sample_2)-1
    homogeneity_p = 1 - st.f.cdf(F, df_1, df_2)

else:
    homogeneity_p = st.kruskal(sample_1, sample_2).pvalue



if (shapiro_1 >= alpha or (shapiro_1 < alpha and len(sample_1) > 30)) and \
 (shapiro_2>= alpha or (shapiro_2 < alpha and len(sample_2)>30)) and \
 homogeneity_p > alpha:

    print('проверяем гипотезу по Стьюденту')
    
    # поскольку наши выборки независимы - используем тест для независимых выборок
    results = st.ttest_ind(sample_1, sample_2)

else:

    print('проверяем гипотезу по Манну-Уитни')
    results = st.mannwhitneyu(sample_1, sample_2)


print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу о равенстве средних двух выборок')
else:
    print('Не получилось отвергнуть нулевую гипотезу о равенстве средних двух выборок') 